# DS100 Final Project: A Basketball Analysis
## Which box score Team Statistics are the best predictors of winning NBA teams?
### Creators: Alec Zhou, Prashant Malyala

## This Project

In this project, we will be investigating which team statistics (measured or calculated by in game performance) are the best predictors of team success in the NBA. For our analysis, we will be using Team Box Score and Standings data from the 2012-2013 through 2017-2018 regular seasons.

## Set up

Below, we will import the necessary libraries for our data analysis.

In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import date

Now, we'll read in the relevant basketball data.

In [111]:
#from the provided ds100 dataset
team_box_score = pd.read_csv('basketball/Basketball-TeamBoxScores.csv')
standings = pd.read_csv('basketball/Standings.csv')

## Data Preview

Let's briefly see what our data looks like.

**Standings Data:**

There are many teams who didn't play on October 30, 2012, who still have their standings listed on that date. Digging deeper, it seems that the Standings data has an entry for every team on every day of each season.

In [112]:
standings.head(5)

,stDate,teamAbbr,rank,rankOrd,gameWon,gameLost,stk,stkType,stkTot,gameBack,...,rel%Indx,mov,srs,pw%,pyth%13.91,wpyth13.91,lpyth13.91,pyth%16.5,wpyth16.5,lpyth16.5
0,2012-10-30,ATL,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
1,2012-10-30,BKN,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
2,2012-10-30,BOS,14,14th,0,1,L1,loss,1,1.0,...,0.0,-13.0,-13.0,0.072,0.1687,13.8334,68.1666,0.131,10.742,71.258
3,2012-10-30,CHA,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
4,2012-10-30,CHI,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000


In [113]:
#30 teams are listed on this date even though not all of them played. there are only 30 teams in the whole NBA
len(standings[standings['stDate'] == '2012-10-30'])

30

Note that much of the information in the Standings table, like streaks or games behind the conference leader, don't classify as "box score team statistics" and therefore aren't relevant to our question. We just want to have a record of how many wins/losses each team had at each point in the season.

**Team Box Score Data:**

For each set of 3 officials, there are 2 games listed on each date. This led us to suspect that the Team Box Score data had two entries per game, one for each team. We can verify this in the second cell below.

In [114]:
team_box_score.head(7)

,gmDate,gmTime,seasTyp,offLNm1,offFNm1,offLNm2,offFNm2,offLNm3,offFNm3,teamAbbr,...,opptFIC40,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,poss,pace
0,2012-10-30,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,WAS,...,61.6667,105.6882,94.4447,11.2435,0.4390,16.7072,1.0476,33.3333,88.9409,88.9409
1,2012-10-30,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,CLE,...,56.0417,94.4447,105.6882,-11.2435,0.3765,18.8679,2.0000,84.6154,88.9409,88.9409
2,2012-10-30,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,BOS,...,80.8333,126.3381,112.6515,13.6866,0.5244,19.8287,3.1250,100.0000,94.9832,94.9832
3,2012-10-30,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,MIA,...,62.7083,112.6515,126.3381,-13.6866,0.4643,18.8501,1.5000,25.0000,94.9832,94.9832
4,2012-10-30,22:30,Regular,Foster,Scott,Zielinski,Gary,Dalen,Eric,DAL,...,58.6458,99.3678,108.1034,-8.7356,0.5000,18.6567,1.7143,42.8571,91.5790,91.5790
5,2012-10-30,22:30,Regular,Foster,Scott,Zielinski,Gary,Dalen,Eric,LAL,...,60.2083,108.1034,99.3678,8.7356,0.4545,17.3338,1.8333,75.0000,91.5790,91.5790
6,2012-10-31,19:00,Regular,Salvatore,Bennett,Brown,Tony,Lewis,Eric,DEN,...,58.4375,89.0099,79.4731,9.5368,0.3448,14.0362,1.1250,81.2500,94.3715,94.3715


In [115]:
#shows that the same game is listed twice, once for each team
team_box_score[team_box_score['gmDate'] == '2012-10-30'][['teamAbbr', 'opptAbbr']]

,teamAbbr,opptAbbr
0,WAS,CLE
1,CLE,WAS
2,BOS,MIA
3,MIA,BOS
4,DAL,LAL
5,LAL,DAL


So not only do we have all the team and opposing team's statistics per game entry, we have two listings of every game! Certainly there's some redundancy in this data, and thinking about how we should address this redundancy will be part of our data cleaning and EDA process.

Note that we have a lot of quantitative variables in the team box score data; these are the statistics we hope to investigate.

As for the categorical data like who the officials were and in what conference the team was, we don't plan to dig into this information as much. However, because it's still worthwhile to see what the implications are of the data we drop, we will have a section dedicated to doing some EDA with this categorical data (refer to the section titled: *Visualizing some features of the Box Score Data*).

## Data Cleaning and Exploratory Data Analysis

### Cleaning our Standings Data

Looking at the Standings.PDF file provided with our data, it's clear that much of the Standings data isn't relevant to us. Our question is chiefly concerned with which **in game** team statistics best predict team success, so external measures of teams' winning/losing streaks, performance relative to the conference, and strength of schedule are all data we can drop.

Note that the assumption that we can predict team success using only team box statistics is inherent to our question. How valid this assumption is will be for us to assess once we build a model.

Here, the measure of success we will refer to is a **team's winning percentage** at different points of the season. 

team winning percentage = $ \frac{W}{W + L} $, where W represents total wins, and L total losses.

In [116]:
#really, we only care about the standing date, team name, and win percentage (computed from wins and losses)
standings = standings[['stDate', 'teamAbbr', 'gameWon', 'gameLost']]

#wpc represents winning percentage
standings['wpc'] = np.divide(standings['gameWon'], standings['gameWon'] + standings['gameLost'])
standings = standings.fillna(0)
standings = standings.drop(['gameWon', 'gameLost'], axis=1)

In [117]:
standings.head()

,stDate,teamAbbr,wpc
0,2012-10-30,ATL,0.0
1,2012-10-30,BKN,0.0
2,2012-10-30,BOS,0.0
3,2012-10-30,CHA,0.0
4,2012-10-30,CHI,0.0


### Merging the Standings and Box Score Data

Now that we have the winning percentages of each team throughout the season from the Standings table, we can inner join it with the Team Box Score data based on the date and team in question.

For every game, we can then see what the winning percentage was of both teams and where their statistical performances differed.

In [118]:
#inner join the tables on the date and teamAbbr so every game has a winning percentage attribute
team_box_score = pd.merge(team_box_score, standings, how='inner',
                          left_on=['gmDate', 'teamAbbr'], right_on=['stDate', 'teamAbbr'])
#drop the added stDate column, which is redundant as we already have a gmDate column
team_box_score = team_box_score.drop('stDate', axis=1)
#and we convert gmDate into a datetime type for ease of manipulation
team_box_score['gmDate'] = team_box_score['gmDate'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

In [119]:
team_box_score.head()

,gmDate,gmTime,seasTyp,offLNm1,offFNm1,offLNm2,offFNm2,offLNm3,offFNm3,teamAbbr,...,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,poss,pace,wpc
0,2012-10-30,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,WAS,...,105.6882,94.4447,11.2435,0.4390,16.7072,1.0476,33.3333,88.9409,88.9409,0.0
1,2012-10-30,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,CLE,...,94.4447,105.6882,-11.2435,0.3765,18.8679,2.0000,84.6154,88.9409,88.9409,1.0
2,2012-10-30,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,BOS,...,126.3381,112.6515,13.6866,0.5244,19.8287,3.1250,100.0000,94.9832,94.9832,0.0
3,2012-10-30,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,MIA,...,112.6515,126.3381,-13.6866,0.4643,18.8501,1.5000,25.0000,94.9832,94.9832,1.0
4,2012-10-30,22:30,Regular,Foster,Scott,Zielinski,Gary,Dalen,Eric,DAL,...,99.3678,108.1034,-8.7356,0.5000,18.6567,1.7143,42.8571,91.5790,91.5790,1.0


### Visualizing some features of the Box Score Data

*Make this section about how officials may affect winning performance, how being the home or away team may affect winning performance, seeing how wins were distributed across each conference/division, and maybe if rest at all played a factor in who was more likely to win a game. We plan to drop all if not most of this data in the next section when we start exploring specific team statistics, so investing some time into exploring the impact of what we're dropping is a good idea.*

In [123]:
def get_nth_season(data, n):
    """1st season begins in 2012 and 6th season begins in 2017"""
    return data[(data['gmDate'] > date(2011 + n, 10, 1)) & (data['gmDate'] < date(2012 + n, 5, 1))]

list_by_season = [get_nth_season(team_box_score, n) for n in range(1,7)]

fig, axs = plt.subplots(3, 2)
for season in list_by_season:
    home_games = season[]
    

[          gmDate gmTime  seasTyp    offLNm1 offFNm1    offLNm2  offFNm2  \
 0     2012-10-30  19:00  Regular   Brothers    Tony      Smith  Michael   
 1     2012-10-30  19:00  Regular   Brothers    Tony      Smith  Michael   
 2     2012-10-30  20:00  Regular  McCutchen   Monty     Wright     Sean   
 3     2012-10-30  20:00  Regular  McCutchen   Monty     Wright     Sean   
 4     2012-10-30  22:30  Regular     Foster   Scott  Zielinski     Gary   
 ...          ...    ...      ...        ...     ...        ...      ...   
 2453  2013-04-17  22:30  Regular      Adams  Bennie      Davis     Marc   
 2454  2013-04-17  22:30  Regular  Zielinski    Gary      Mauer      Ken   
 2455  2013-04-17  22:30  Regular  Zielinski    Gary      Mauer      Ken   
 2456  2013-04-17  22:30  Regular       Wood    Leon     Palmer   Violet   
 2457  2013-04-17  22:30  Regular       Wood    Leon     Palmer   Violet   
 
          offLNm3   offFNm3 teamAbbr  ...  opptOrtg  opptDrtg opptEDiff  \
 0        W

### Cleaning our Team Box Score Data

Our box score data has a lot of columns, making it quite difficult to find patterns. Not all of the columns are directly relevant to the question we're trying to answer, so let's see what we can clean up.

Just in case we want to revisit any of the team_box_score data, we'll store our changes in different dataframes.

First of all, the time of each game and the identities of our officials aren't really team statistics. We may choose to revisit them later, but for now, let's ignore them.

In [ ]:
#note that none of these seasons were lockouts or out of the ordinary. this seasTyp column also doesn't tell us much
team_box_score['seasTyp'].unique()

In [ ]:
#dropping the aforementioned columns
tbs_cleaned = team_box_score.drop(['gmTime', 'seasTyp', 'offLNm1', 'offFNm1',
                                   'offLNm2', 'offFNm2', 'offLNm3', 'offFNm3'], axis=1)

The conference and division a team plays in, the number of days they've had off, and the total team minutes (which would be the same for both teams in a game) also aren't really performance statistics. We will also drop this information for now.

Note that we will also drop this data for the opponent side (opptConf, opptDiv, etc.).

In [ ]:
#dropping the columns we've mentioned above
tbs_cleaned = tbs_cleaned.drop(['teamConf', 'teamDiv', 'teamMin', 'teamDayOff', 'opptConf',
                                  'opptDiv', 'opptMin', 'opptDayOff'], axis=1)

In [ ]:
tbs_cleaned.head()

Finally, just to make the teamRslt column a bit more computer interpretable, let's replace losses with 0's and wins with 1's.

In [ ]:
#make the result column a binary indicator
tbs_cleaned = tbs_cleaned.replace(to_replace=['Loss', 'Win'], value=[0, 1])
#verify that we only have 0's and 1's in this column
tbs_cleaned['teamRslt'].unique()

In [ ]:
tbs_cleaned.head()

We've narrowed our data mostly down to the barebones numbers and statistics—awesome!

Now we can focus on exploring our data a bit more thoroughly.

### Exploring the Team Box Score Data

In [ ]:
np.var(tbs_cleaned).sort_values(ascending=False)[0:25]

In [ ]:
numbers_only = tbs_cleaned.drop(['teamAbbr', 'teamLoc', 'opptAbbr', 'opptConf', 'opptDiv', 'opptLoc', 'opptRslt'], axis=1)

In [ ]:
#centers data mean on 0, can now do SVD
numbers_only_centered = numbers_only - np.mean(numbers_only)
numbers_only_centered.head()

In [ ]:
#divide by standard deviation to prevent high variance components from affecting PCA
numbers_only_centered_scaled = np.divide(numbers_only_centered, np.sqrt(np.var(numbers_only_centered)))
numbers_only_centered_scaled.head()

In [ ]:
u, s, vt = np.linalg.svd(numbers_only_centered_scaled, full_matrices=False)
p_matrix = numbers_only_centered_scaled @ vt.T

In [ ]:
plt.plot(np.arange(1, 104), s**2 / sum(s**2));
plt.xticks(np.arange(1, 104), np.arange(1, 104));
plt.xlabel('PC #');
plt.ylabel('Variance Explained');
plt.title('Scree Plot of Variance of PC #i')

In [ ]:
numbers_only_filtered = numbers_only[numbers_only.columns.drop(list(numbers_only.filter(regex='oppt')))]

df_rando = numbers_only_filtered[['teamOREB%', 'teamEDiff', 'teamSTL', 'teamBLK', 'teamFG%', 'team3P%', 'teamORB',
                                 'teamDRB', 'teamPTS3', 'teamPTS4', 'teamASST%', 'teamEFG%', 'teamOrtg', 'teamDrtg',
                                 'teamSTL/TO']]
df_rando.head()

In [ ]:
sns.heatmap(df_rando.corr())

## Inference and Prediction